Add in return flights for round-trips; summarize frequencies by market

## imports

In [1]:
import datetime

import pyspark.sql.functions as F
import pyspark.sql.types as T 
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1672772170539_0002,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## load data

In [2]:
BASE_INPUT_DIR = "s3://tvlp-ds-users/kendra-frederick/shopping-grid/agg-raw-data_v4/*"
input_dir = BASE_INPUT_DIR

df = spark.read.parquet(input_dir)
# df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# df.printSchema()

# df.columns

df.select(F.min("shop_date"), F.max('shop_date')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+--------------+
|min(shop_date)|max(shop_date)|
+--------------+--------------+
|      20221110|      20221231|
+--------------+--------------+

In [4]:
# # if we want to impose any limits on date range
# date_llim = 20221110
# date_ulim = 20221213
# df_filt = df.filter(F.col("shop_date").between(date_llim, date_ulim)).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# apply fare rank filter here (so it's easier to modify in the future)
# df_filt = df_filt.filter(F.col("fare_rank") == 1)
df_filt = df.filter(F.col("fare_rank") == 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

A peek at the data:

In [6]:
df_filt.select('shop_date', 'market', 'out_departure_date', 'out_marketing_cxr', 'out_flight_numbers', 'in_marketing_cxr', 'adt_fare', 'fare_rank',).show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------+------------------+-----------------+------------------+----------------+--------+---------+
|shop_date| market|out_departure_date|out_marketing_cxr|out_flight_numbers|in_marketing_cxr|adt_fare|fare_rank|
+---------+-------+------------------+-----------------+------------------+----------------+--------+---------+
| 20221229|ATL-EWR|          20221229|             [DL]|             [400]|            [DL]|   730.2|        1|
| 20221229|ATL-EWR|          20221230|             [UA]|            [1829]|        [UA, UA]|  737.92|        1|
| 20221229|ATL-EWR|          20230104|             [UA]|             [222]|            [UA]|  285.82|        1|
| 20221229|ATL-EWR|          20230104|         [AA, AA]|      [2174, 2718]|        [AA, AA]|  242.19|        1|
| 20221229|ATL-EWR|          20230104|         [AA, AA]|      [2174, 2718]|        [AA, AA]|  242.19|        1|
| 20221229|ATL-EWR|          20230104|         [AA, AA]|      [2129, 1216]|        [AA, AA]|  242.19|   

## add date-based features

In [7]:
# convert dates (which are ints) to datetime
df_filt.registerTempTable("data")
df_filt = spark.sql("""
    SELECT *,
        TO_DATE(CAST(UNIX_TIMESTAMP(CAST(out_departure_date AS string), 'yyyyMMdd') AS TIMESTAMP)) AS outDeptDt_dt,
        TO_DATE(CAST(UNIX_TIMESTAMP(CAST(in_departure_date AS string), 'yyyyMMdd') AS TIMESTAMP)) AS inDeptDt_dt,
        TO_DATE(CAST(UNIX_TIMESTAMP(CAST(shop_date AS string), 'yyyyMMdd') AS TIMESTAMP)) AS searchDt_dt
    FROM data
""")

# add days_til_dept column
df_filt = df_filt.withColumn('days_til_dept', F.datediff(
                    F.col('outDeptDt_dt'), F.col('searchDt_dt'))
                )

## add in DOW indicators
def add_dow_ind(df):
    df = (df
        .withColumn("out_dt_dow_int", (F.date_format("outDeptDt_dt", "u") - 1).cast(T.IntegerType()))
        .withColumn("out_dt_dow", F.date_format("outDeptDt_dt", "E"))
        .withColumn("in_dt_dow_int", (F.date_format("inDeptDt_dt", "u") - 1).cast(T.IntegerType()))
        .withColumn("in_dt_dow", F.date_format("inDeptDt_dt", "E"))
        )
    return df

df_filt = add_dow_ind(df_filt)
df_filt = df_filt.filter(F.col("days_til_dept") > 0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_filt.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- market: string (nullable = true)
 |-- out_origin_airport: string (nullable = true)
 |-- out_destination_airport: string (nullable = true)
 |-- out_departure_date: long (nullable = true)
 |-- in_departure_date: long (nullable = true)
 |-- pcc: string (nullable = true)
 |-- gds: string (nullable = true)
 |-- shop_req_timeStamp: long (nullable = true)
 |-- point_of_sale: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- out_origin_city: string (nullable = true)
 |-- out_destination_city: string (nullable = true)
 |-- constricted_search: boolean (nullable = true)
 |-- out_marketing_cxr: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- out_departure_time: integer (nullable = true)
 |-- out_flight_numbers: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- out_booking_class: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- out_cabin_class: array (nullable = true)
 |    |-- element

# Notes on processing

Notes:
- eventually, we want to analyze:
    - out-bound one-way (out_ow)
    - out-bound round-trip (out_rt)
    - in-bound round-trip (in_rt)
    - round-trip (out+in) (rt)
- first, process by out-bound and in-bound (rt only)
    - out later gets split into ow & rt
- include `round_trip` field in in-bound groupby's
    - not necessary, but simplifies subsequent processing
    

# Analysis by carrier

Notes:
- Crude: list of carriers, no matter if they provide direct lowest-fare flight or not
- Less Crude: combine interline carriers into single string

## common prep

In [9]:
# extract carrier info
df_cxr = (df_filt
          .withColumn("out_unique_cxr", F.array_distinct("out_marketing_cxr"))
          .withColumn("out_cxr_all_same", F.when(F.size("out_unique_cxr") == 1, 1).otherwise(0))
          .withColumn("in_unique_cxr", F.array_distinct("in_marketing_cxr"))
          .withColumn("in_cxr_all_same", F.when(F.size("in_unique_cxr") == 1, 1).otherwise(0))
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Explode used for both crude & nuanced

In [10]:
# Note we use `explode_outer` for in-bound carriers so nulls don't get dropped
out_cxr_expl = (df_cxr
               .select("*", F.explode("out_unique_cxr").alias("out_cxr"))
              .withColumn("out_uni_cxr_concat", F.concat_ws("-", "out_unique_cxr"))
              )

in_cxr_expl = (df_cxr
               .select("*", F.explode_outer("in_unique_cxr").alias("in_cxr"))
              .withColumn("in_uni_cxr_concat", F.concat_ws("-", "in_unique_cxr"))
              )

# could merge exploded df's and concat those, but not worth the trouble (for now)
rt_cxr = (df_cxr
          .withColumn("rt_cxrs", F.concat_ws("_", "out_unique_cxr", "in_unique_cxr"))
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## group-by-agg's

### "crude" (single carrier)
regardless of num stops / inter-line

In [11]:
# new to 02b: add DOW, DTD
base_grpby_cols = ["market", "round_trip", "days_til_dept", "out_dt_dow_int", "in_dt_dow_int"]
out_cxr_crude = (
    out_cxr_expl
    .groupBy(base_grpby_cols + ["out_cxr"])
    .agg(F.count("*").alias("count"))
)
rt_in_cxr_crude = (
    in_cxr_expl
    .filter(F.col("round_trip") == 1)
    .groupBy(base_grpby_cols + ["in_cxr"])
    .agg(F.count("*").alias("count"))
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### "nuanced" (inter-line)


In [12]:
# exclude num stops for now
out_cxr_nuanced = (
    out_cxr_expl
    .groupBy(base_grpby_cols + ["out_uni_cxr_concat"]) # "out_num_stops",
    .agg(F.count("*").alias("count"))
)
rt_in_cxr_nuanced = (
    in_cxr_expl
    .filter(F.col("round_trip") == 1)
    .groupBy(base_grpby_cols + ["in_uni_cxr_concat"]) # "in_num_stops",
    .agg(F.count("*").alias("count"))
)
rt_cxr_nuanced = (
    rt_cxr
    .filter(F.col("round_trip") == 1)
    .groupBy(base_grpby_cols + ["rt_cxrs"])
    .agg(F.count("*").alias("count"))
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## window funcs
- to calc pct by market

In [13]:
# use a single window func to make it easy
w1 = Window.partitionBy("market", "round_trip")

def apply_window_1(df, w):
    df = (df
          .withColumn("count_market", F.sum("count").over(w))
          .withColumn("pct_count", F.col("count") / F.col("count_market"))
         )
    return df

w2 = (Window
      .partitionBy("market", "round_trip")
      .orderBy(F.desc("pct_count"))
      .rowsBetween(Window.unboundedPreceding, Window.currentRow)
     )

def apply_window_2(df, w2):
    df = df.withColumn("cum_pct", F.sum("pct_count").over(w2))
    return df

out_cxr_crude = apply_window_1(out_cxr_crude, w1)
out_cxr_crude = apply_window_2(out_cxr_crude, w2)

rt_in_cxr_crude = apply_window_1(rt_in_cxr_crude, w1)
rt_in_cxr_crude = apply_window_2(rt_in_cxr_crude, w2)


out_cxr_nuanced = apply_window_1(out_cxr_nuanced, w1)
out_cxr_nuanced = apply_window_2(out_cxr_nuanced, w2)

rt_in_cxr_nuanced = apply_window_1(rt_in_cxr_nuanced, w1)
rt_in_cxr_nuanced = apply_window_2(rt_in_cxr_nuanced, w2)

rt_cxr_nuanced = apply_window_1(rt_cxr_nuanced, w1)
rt_cxr_nuanced = apply_window_2(rt_cxr_nuanced, w2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# split 'out' into ow & rt
ow_out_cxr_crude = out_cxr_crude.filter(F.col("round_trip") == 0)
rt_out_cxr_crude = out_cxr_crude.filter(F.col("round_trip") == 1)

ow_out_cxr_nuanced = out_cxr_nuanced.filter(F.col("round_trip") == 0)
rt_out_cxr_nuanced = out_cxr_nuanced.filter(F.col("round_trip") == 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## save
plotting will be easier done locally

In [15]:
# ow_out_cxr_crude.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
if 1 == 0:
    OUTPUT_DIR = "s3://tvlp-ds-users/kendra-frederick/shopping-grid/carrier-consistency/"

    ow_out_cxr_crude.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "ow_out_cxr_crude", header=True)
    rt_out_cxr_crude.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "rt_out_cxr_crude", header=True)
    rt_in_cxr_crude.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "rt_in_cxr_crude", header=True)

    ow_out_cxr_nuanced.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "ow_out_cxr_nuanced", header=True)
    rt_out_cxr_nuanced.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "rt_out_cxr_nuanced", header=True)
    rt_in_cxr_nuanced.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "rt_in_cxr_nuanced", header=True)
    
    rt_cxr_nuanced.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "rt_cxr_nuanced", header=True)    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Analysis by flight

## common prep

In [17]:
# fi = flight info = carrier + flight no

# combine cxr + flight no into array of concat'd strings
df_fi = (df_filt
       .withColumn("out_flight_nos_str", F.col("out_flight_numbers").cast('array<string>'))
       .withColumn("out_fi_zip", F.arrays_zip(F.col("out_marketing_cxr"), F.col("out_flight_nos_str")))
       .withColumn('out_fi_zip_concat', F.expr("transform(out_fi_zip, x -> concat_ws('-', x.out_marketing_cxr, x.out_flight_nos_str))"))
       .withColumn("in_flight_nos_str", F.col("in_flight_numbers").cast('array<string>'))
       .withColumn("in_fi_zip", F.arrays_zip(F.col("in_marketing_cxr"), F.col("in_flight_nos_str")))
       .withColumn('in_fi_zip_concat', F.expr("transform(in_fi_zip, x -> concat_ws('-', x.in_marketing_cxr, x.in_flight_nos_str))"))
      )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# df_fi.select("out_marketing_cxr", "out_flight_numbers", "out_fi_zip_concat").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## group-by-agg

### crude (single leg)

In [19]:
# # each leg separately (explode cols --> rows)
# df_fi_expl = (df_fi
#            .select("*", F.posexplode("out_fi_zip_concat").alias("out_leg_num", "out_fi_leg"))
#            .withColumn("out_leg_num", F.col("out_leg_num") + 1)
#            .select("*", F.posexplode_outer("in_fi_zip_concat").alias("in_leg_num", "in_fi_leg"))
#            .withColumn("in_leg_num", F.col("in_leg_num") + 1)
#           )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# df_fi_expl.select("market", "shop_req_timeStamp", "out_marketing_cxr", "out_flight_numbers", "out_fi_zip_concat",
#                  "out_fi_leg", "out_leg_num").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
 df_fi_expl.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_fi_expl' is not defined
Traceback (most recent call last):
NameError: name 'df_fi_expl' is not defined



In [22]:
df_fi_expl_no_dup = df_fi_expl.drop_duplicates()
df_fi_expl_no_dup.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_fi_expl' is not defined
Traceback (most recent call last):
NameError: name 'df_fi_expl' is not defined



Thos are not duplicates

In [23]:
df_fi_expl.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'df_fi_expl' is not defined
Traceback (most recent call last):
NameError: name 'df_fi_expl' is not defined



Cause is the combination of exploding out + in

In [24]:
# each leg separately (explode cols --> rows)
out_fi_expl = (df_fi
           .select("*", F.posexplode("out_fi_zip_concat").alias("out_leg_num", "out_fi_leg"))
           .withColumn("out_leg_num", F.col("out_leg_num") + 1)
          )

in_fi_expl = (df_fi
           .select("*", F.posexplode_outer("in_fi_zip_concat").alias("in_leg_num", "in_fi_leg"))
           .withColumn("in_leg_num", F.col("in_leg_num") + 1)
          )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# define above; copied here for reference
# base_grpby_cols = ["market", "round_trip", "days_til_dept", "out_dt_dow_int", "in_dt_dow_int"]

out_fi_per_leg = (out_fi_expl
                    .groupBy(base_grpby_cols + ["out_fi_leg"])
                    .agg(F.count("*").alias("count"))
                 )

rt_in_fi_per_leg = (in_fi_expl
                    .filter(F.col("round_trip") == 1)
                    .groupBy(base_grpby_cols + ["in_fi_leg"])
                    .agg(F.count("*").alias("count"))
                 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### nuanced (all legs --> one string)

In [26]:
# combine mulit-legs into single string
df_fi_one_str = (df_fi
              .withColumn("out_fi_itin", F.concat_ws(".", "out_fi_zip_concat"))
              .withColumn("in_fi_itin", F.concat_ws(".", "in_fi_zip_concat"))
              .withColumn("rt_fi_itin", F.concat_ws("_", "out_fi_itin", "in_fi_itin"))
             )

out_fi_one_str = (df_fi_one_str
                      .groupBy(base_grpby_cols + ["out_fi_itin"])
                      .agg(F.count("*").alias("count"))
                     )

rt_in_fi_one_str = (df_fi_one_str
                      .filter(F.col("round_trip") == 1)
                      .groupBy(base_grpby_cols + ["in_fi_itin"])
                      .agg(F.count("*").alias("count"))
                     )

rt_fi_one_str = (df_fi_one_str
                      .filter(F.col("round_trip") == 1)
                      .groupBy(base_grpby_cols + ["rt_fi_itin"])
                      .agg(F.count("*").alias("count"))
                     )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## window funcs

In [27]:
out_fi_per_leg = apply_window_1(out_fi_per_leg, w1)
out_fi_per_leg = apply_window_2(out_fi_per_leg, w2)

rt_in_fi_per_leg = apply_window_1(rt_in_fi_per_leg, w1)
rt_in_fi_per_leg = apply_window_2(rt_in_fi_per_leg, w2)

out_fi_one_str = apply_window_1(out_fi_one_str, w1)
out_fi_one_str = apply_window_2(out_fi_one_str, w2)

rt_in_fi_one_str = apply_window_1(rt_in_fi_one_str, w1)
rt_in_fi_one_str = apply_window_2(rt_in_fi_one_str, w2)

rt_fi_one_str = apply_window_1(rt_fi_one_str, w1)
rt_fi_one_str = apply_window_2(rt_fi_one_str, w2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# split into ow & rt
ow_out_fi_per_leg = out_fi_per_leg.filter(F.col("round_trip") == 0)
rt_out_fi_per_leg = out_fi_per_leg.filter(F.col("round_trip") == 1)

ow_out_fi_one_str = out_fi_one_str.filter(F.col("round_trip") == 0)
rt_out_fi_one_str = out_fi_one_str.filter(F.col("round_trip") == 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## save

UPDATE THIS

In [31]:
if 1 == 0:
    OUTPUT_DIR = "s3://tvlp-ds-users/kendra-frederick/shopping-grid/carrier-consistency/"

    ow_out_fi_per_leg.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "ow_out_fi_per_leg", header=True)
    rt_out_fi_per_leg.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "rt_out_fi_per_leg", header=True)
    rt_in_fi_per_leg.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "rt_in_fi_per_leg", header=True)

    ow_out_fi_one_str.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "ow_out_fi_one_str", header=True)
    rt_out_fi_one_str.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "rt_out_fi_one_str", header=True)
    rt_in_fi_one_str.coalesce(1).write.mode("overwrite").csv(OUTPUT_DIR + "rt_in_fi_one_str", header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Dev analysis

For market overview, we want to agg-out the DTD and DOW dimensions

In [30]:
market = 'ATL-EWR'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
mdf = ow_out_cxr_crude.filter(F.col("market") == market)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
agg_df = (mdf
 .groupBy('round_trip', 'out_cxr')
 .agg(
     F.sum("count").alias('count'),
     F.sum("pct_count").alias('pct_count')
 )
         )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
agg_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+------+--------------------+
|round_trip|out_cxr| count|           pct_count|
+----------+-------+------+--------------------+
|         0|     NK|388702|  0.5377822235319661|
|         0|     AS|     6|8.301200768691191E-6|
|         0|     AA|  9184|0.012706371309943319|
|         0|     B6|   130|1.798593499883090...|
|         0|     UA|198977| 0.27529133755864377|
|         0|     DL|125788| 0.17403190704868776|
+----------+-------+------+--------------------+

In [60]:
cum_agg_df = apply_window_2(agg_df, w2)
cum_agg_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
"cannot resolve '`market`' given input columns: [data.round_trip, out_cxr, count, pct_count];;\n'Project [round_trip#25, out_cxr#2975, count#5162L, pct_count#5164, sum(pct_count#5164) windowspecdefinition('market, round_trip#25, pct_count#5164 DESC NULLS LAST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS cum_pct#5193]\n+- Aggregate [round_trip#25, out_cxr#2975], [round_trip#25, out_cxr#2975, sum(count#3250L) AS count#5162L, sum(pct_count#3487) AS pct_count#5164]\n   +- Filter (market#0 = ATL-EWR)\n      +- Filter (round_trip#25 = 0)\n         +- Project [market#0, round_trip#25, days_til_dept#151, out_dt_dow_int#187, in_dt_dow_int#262, out_cxr#2975, count#3250L, count_market#3478L, pct_count#3487, cum_pct#3498]\n            +- Project [market#0, round_trip#25, days_til_dept#151, out_dt_dow_int#187, in_dt_dow_int#262, out_cxr#2975, count#3250L, count_market#3478L, pct_count#3487, cum_pct#3498, cum_pct#3498]\n               +- Window 

In [ ]:
(ow_out_cxr_crude
 .filter(F.col("market") == market)
 .groupBy("
 .drop("count_market").orderBy(F.desc("pct_count")).show()
    )

# Results by market

For market overview, we want to agg-out the DTD and DOW dimensions

In [32]:
ow_out_cxr_crude_agg = (ow_out_cxr_crude
                        .groupBy('market', 'round_trip', 'out_cxr')
                        .agg(
                             F.sum("count").alias('count'),
                             F.sum("pct_count").alias('pct_count')
                         )
                        .withColumn("cum_pct", F.sum("pct_count").over(w2))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
ow_out_cxr_nuanced_agg = (ow_out_cxr_nuanced
                        .groupBy('market', 'round_trip', 'out_uni_cxr_concat')
                        .agg(
                             F.sum("count").alias('count'),
                             F.sum("pct_count").alias('pct_count')
                         )
                        .withColumn("cum_pct", F.sum("pct_count").over(w2))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
rt_out_cxr_nuanced_agg = (rt_out_cxr_nuanced
                        .groupBy('market', 'round_trip', 'out_uni_cxr_concat')
                        .agg(
                             F.sum("count").alias('count'),
                             F.sum("pct_count").alias('pct_count')
                         )
                        .withColumn("cum_pct", F.sum("pct_count").over(w2))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
rt_in_cxr_nuanced_agg = (rt_in_cxr_nuanced
                        .groupBy('market', 'round_trip', 'in_uni_cxr_concat')
                        .agg(
                             F.sum("count").alias('count'),
                             F.sum("pct_count").alias('pct_count')
                         )
                        .withColumn("cum_pct", F.sum("pct_count").over(w2))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
rt_cxr_nuanced_agg = (rt_cxr_nuanced
                        .groupBy('market', 'round_trip', 'rt_cxrs')
                        .agg(
                             F.sum("count").alias('count'),
                             F.sum("pct_count").alias('pct_count')
                         )
                        .withColumn("cum_pct", F.sum("pct_count").over(w2))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
ow_out_fi_per_leg_agg = (ow_out_fi_per_leg
                        .groupBy('market', 'round_trip', 'out_fi_leg')
                        .agg(
                             F.sum("count").alias('count'),
                             F.sum("pct_count").alias('pct_count')
                         )
                        .withColumn("cum_pct", F.sum("pct_count").over(w2))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
rt_out_fi_per_leg_agg = (rt_out_fi_per_leg
                        .groupBy('market', 'round_trip', 'out_fi_leg')
                        .agg(
                             F.sum("count").alias('count'),
                             F.sum("pct_count").alias('pct_count')
                         )
                        .withColumn("cum_pct", F.sum("pct_count").over(w2))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
rt_in_fi_per_leg_agg = (rt_in_fi_per_leg
                        .groupBy('market', 'round_trip', 'in_fi_leg')
                        .agg(
                             F.sum("count").alias('count'),
                             F.sum("pct_count").alias('pct_count')
                         )
                        .withColumn("cum_pct", F.sum("pct_count").over(w2))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
ow_out_fi_one_str_agg = (ow_out_fi_one_str
                        .groupBy('market', 'round_trip', 'out_fi_itin')
                        .agg(
                             F.sum("count").alias('count'),
                             F.sum("pct_count").alias('pct_count')
                         )
                        .withColumn("cum_pct", F.sum("pct_count").over(w2))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
rt_out_fi_one_str_agg = (rt_out_fi_one_str
                        .groupBy('market', 'round_trip', 'out_fi_itin')
                        .agg(
                             F.sum("count").alias('count'),
                             F.sum("pct_count").alias('pct_count')
                         )
                        .withColumn("cum_pct", F.sum("pct_count").over(w2))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
rt_in_fi_one_str_agg = (rt_in_fi_one_str
                        .groupBy('market', 'round_trip', 'in_fi_itin')
                        .agg(
                             F.sum("count").alias('count'),
                             F.sum("pct_count").alias('pct_count')
                         )
                        .withColumn("cum_pct", F.sum("pct_count").over(w2))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
rt_fi_one_str_agg = (rt_fi_one_str
                        .groupBy('market', 'round_trip', 'rt_fi_itin')
                        .agg(
                             F.sum("count").alias('count'),
                             F.sum("pct_count").alias('pct_count')
                         )
                        .withColumn("cum_pct", F.sum("pct_count").over(w2))
                       )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
def show_market_analysis(market):
    # For market overview, we want to agg-out the DTD and DOW dimensions
    
    print("Carriers - any leg - one-way")
    (ow_out_cxr_crude_agg
     .filter(F.col("market") == market)
     .orderBy(F.desc("pct_count")).show()
    )
    
    # print("Carriers - any leg - Round-trip")
    # same_cxr_crude.filter(F.col("market") == market).drop("count_market").orderBy(F.desc("pct_count")).show()
    
    print("Carriers - incl. interline - one-way (out-bound)")
    (ow_out_cxr_nuanced_agg
     .filter(F.col("market") == market)
     .orderBy(F.desc("pct_count"))
     .show()
    )

    print("Carriers - incl. interline - round-trip, out-bound")
    (rt_out_cxr_nuanced_agg
     .filter(F.col("market") == market)
     .orderBy(F.desc("pct_count"))
     .show()
    )

    print("Carriers - incl. interline - round-trip, in-bound")
    (rt_in_cxr_nuanced_agg
     .filter(F.col("market") == market)
     .orderBy(F.desc("pct_count"))
     .show()
    )
    
    print("Carriers - incl. interline - round-trip")
    (rt_cxr_nuanced_agg
     .filter(F.col("market") == market)
     .orderBy(F.desc("pct_count"))
     .show()
    )  
    
    print("Individual Legs - one-way (out-bound)")
    (ow_out_fi_per_leg_agg
     .filter(F.col("market") == market)
     .orderBy(F.desc("count"))
     .show()
    )
    
    print("Individual Legs - round-trip, out-bound")
    (rt_out_fi_per_leg_agg
     .filter(F.col("market") == market)
     .orderBy(F.desc("count"))
     .show()
    )
    
    print("Individual Legs - round-trip, in-bound")
    (rt_in_fi_per_leg_agg
     .filter(F.col("market") == market)
     .orderBy(F.desc("count"))
     .show()
    )
    
    print("Itineraries - one-way (out-bound)")
    (ow_out_fi_one_str_agg
     .filter(F.col("market") == market)
     .orderBy(F.desc("pct_count"))
     .show()
    )

    print("Itineraries - round-trip, out-bound")
    (rt_out_fi_one_str_agg
     .filter(F.col("market") == market)
     .orderBy(F.desc("pct_count"))
     .show(truncate=False)
    )
    
    print("Itineraries - round-trip, in-bound")
    (rt_in_fi_one_str_agg
     .filter(F.col("market") == market)
     .orderBy(F.desc("pct_count"))
     .show(truncate=False)
    )
    
    print("Itineraries - round-trip, out+in")
    (rt_fi_one_str_agg
     .filter(F.col("market") == market)
     .orderBy(F.desc("pct_count"))
     .show(truncate=False)
    )
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## JFK-LHR

In [92]:
show_market_analysis("JFK-LHR")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Carriers - any leg - one-way
+-------+----------+-------+-------+--------------------+------------------+
| market|round_trip|out_cxr|  count|           pct_count|           cum_pct|
+-------+----------+-------+-------+--------------------+------------------+
|JFK-LHR|         0|     TP|1553417|  0.8271814582175185|0.8271814582175185|
|JFK-LHR|         0|     DL| 125581| 0.06687082393485705|0.8940522821523755|
|JFK-LHR|         0|     VS| 119529| 0.06364818494923222|0.9577004671016077|
|JFK-LHR|         0|     WS|  47143| 0.02510325011555064|0.9828037172171583|
|JFK-LHR|         0|     FI|  22945|0.012218019088757887|0.9950217363059162|
|JFK-LHR|         0|     SK|   1847|9.835119310061336E-4|0.9960052482369223|
|JFK-LHR|         0|     TK|   1584|8.434666479229633E-4|0.9968487148848453|
|JFK-LHR|         0|     AF|   1221|6.501722077739509E-4|0.9974988870926192|
|JFK-LHR|         0|     AZ|   1148|6.113003231158851E-4|0.9981101874157351|
|JFK-LHR|         0|     JU|    718|3.823289477

## LHR-JFK

In [93]:
show_market_analysis("LHR-JFK")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Carriers - any leg - one-way
+-------+----------+-------+------+--------------------+------------------+
| market|round_trip|out_cxr| count|           pct_count|           cum_pct|
+-------+----------+-------+------+--------------------+------------------+
|LHR-JFK|         0|     TP|708534|  0.9174906636693692|0.9174906636693692|
|LHR-JFK|         0|     FI| 26689| 0.03455996229210156|0.9520506259614707|
|LHR-JFK|         0|     DE|  9596|0.012425995659447957|0.9644766216209186|
|LHR-JFK|         0|     LH|  9596|0.012425995659447955|0.9769026172803665|
|LHR-JFK|         0|     TK|  8794|0.011387474554938033|0.9882900918353046|
|LHR-JFK|         0|     WS|  3657|0.004735500846873821|0.9930255926821784|
|LHR-JFK|         0|     AT|  2764|0.003579142559682...| 0.996604735241861|
|LHR-JFK|         0|     S4|  1988|0.002574289221653...| 0.999179024463514|
|LHR-JFK|         0|     EI|   355|4.596945038666136...|0.9996387189673807|
|LHR-JFK|         0|     JU|   104|1.346710659214867...|0.9

## ATL-EWR

In [94]:
show_market_analysis("ATL-EWR")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Carriers - any leg - one-way
+-------+----------+-------+------+--------------------+------------------+
| market|round_trip|out_cxr| count|           pct_count|           cum_pct|
+-------+----------+-------+------+--------------------+------------------+
|ATL-EWR|         0|     NK|388702|  0.5377822235319664|0.5377822235319664|
|ATL-EWR|         0|     UA|198977| 0.27529133755864355|0.8130735610906099|
|ATL-EWR|         0|     DL|125788| 0.17403190704868776|0.9871054681392977|
|ATL-EWR|         0|     AA|  9184|0.012706371309943308| 0.999811839449241|
|ATL-EWR|         0|     B6|   130|1.798593499883091...|0.9999916987992293|
|ATL-EWR|         0|     AS|     6|8.301200768691191E-6| 0.999999999999998|
+-------+----------+-------+------+--------------------+------------------+

Carriers - incl. interline - one-way (out-bound)
+-------+----------+------------------+------+--------------------+------------------+
| market|round_trip|out_uni_cxr_concat| count|           pct_count|       

## SFO-LAX

In [ ]:
market = 'SFO-LAX'
show_market_analysis(market)

## LGA-MIA

In [ ]:
market = 'LGA-MIA'
show_market_analysis(market)

## OAK-LAS

In [ ]:
market = 'OAK-LAS'
show_market_analysis(market)

## LAX-SFO

In [ ]:
market = 'LAX-SFO'
show_market_analysis(market)

## LAX-JFK

In [ ]:
market = 'LAX-JFK'
show_market_analysis(market)

## JFK-LAX

In [ ]:
market = 'JFK-LAX'
show_market_analysis(market)

## EWR-CDG

In [ ]:
market = 'EWR-CDG'
show_market_analysis(market)

## others

In [ ]:
market_one_str_grp.select("market").distinct().show()

# Higher level summary

## num to get to X %

In [96]:
def count_num_records_to_cutoff(df, cutoff, things="things"):
    w3 = Window.partitionBy("market").orderBy('cum_pct')

    window_df = df.withColumn("row_num", F.row_number().over(w3))
    filt_df = window_df.filter(F.col("cum_pct") <= cutoff)
    # could add a collect_list agg here 
    final_result = filt_df.groupBy("market").agg(F.max("row_num").alias(f"num_{things}"))
    final_result.orderBy("market").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
cutoff = 0.99

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
count_num_records_to_cutoff(ow_out_cxr_crude_agg, cutoff, "carriers")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+
| market|num_carriers|
+-------+------------+
|ATL-EWR|           3|
|EWR-CDG|           8|
|EWR-LHR|           4|
|JFK-LAX|           2|
|JFK-LHR|           4|
|LAX-EWR|           5|
|LAX-JFK|           2|
|LAX-SFO|           3|
|LGA-MIA|           3|
|LHR-EWR|           2|
|LHR-JFK|           5|
|LHR-LAX|           8|
|OAK-LAS|           2|
|SFO-LAX|           4|
+-------+------------+

Num carriers, round-trip, including inter-line

In [99]:
count_num_records_to_cutoff(rt_out_cxr_nuanced_agg, cutoff, "carriers")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+
| market|num_carriers|
+-------+------------+
|ATL-EWR|           2|
|EWR-CDG|          17|
|EWR-LHR|          15|
|JFK-LAX|           2|
|JFK-LHR|          18|
|LAX-EWR|           4|
|LAX-JFK|           2|
|LAX-SFO|           4|
|LGA-MIA|           3|
|LHR-EWR|          10|
|LHR-JFK|          12|
|LHR-LAX|          22|
|OAK-LAS|           3|
|SFO-LAX|           4|
+-------+------------+

Num legs - one-way

In [100]:
cutoff = 0.90

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [101]:
count_num_records_to_cutoff(ow_out_fi_per_leg_agg, cutoff, "legs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
| market|num_legs|
+-------+--------+
|ATL-EWR|      24|
|EWR-CDG|      25|
|EWR-LHR|       8|
|JFK-LAX|      25|
|JFK-LHR|       9|
|LAX-EWR|      18|
|LAX-JFK|      36|
|LAX-SFO|      31|
|LGA-MIA|      12|
|LHR-EWR|       7|
|LHR-JFK|       4|
|LHR-LAX|      25|
|OAK-LAS|      11|
|SFO-LAX|      36|
+-------+--------+

Num legs - round-trip, out-bound

In [102]:
count_num_records_to_cutoff(rt_out_fi_per_leg_agg, cutoff, "legs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
| market|num_legs|
+-------+--------+
|ATL-EWR|      40|
|EWR-CDG|      46|
|EWR-LHR|      75|
|JFK-LAX|      33|
|JFK-LHR|      69|
|LAX-EWR|      76|
|LAX-JFK|      36|
|LAX-SFO|      42|
|LGA-MIA|      20|
|LHR-EWR|      49|
|LHR-JFK|      54|
|LHR-LAX|     169|
|OAK-LAS|      30|
|SFO-LAX|      42|
+-------+--------+

Num legs - round-trip, in-bound

In [103]:
count_num_records_to_cutoff(rt_in_fi_per_leg_agg, cutoff, "legs")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
| market|num_legs|
+-------+--------+
|ATL-EWR|      40|
|EWR-CDG|      44|
|EWR-LHR|      89|
|JFK-LAX|      35|
|JFK-LHR|      72|
|LAX-EWR|      74|
|LAX-JFK|      32|
|LAX-SFO|      40|
|LGA-MIA|      23|
|LHR-EWR|      47|
|LHR-JFK|      33|
|LHR-LAX|     110|
|OAK-LAS|      25|
|SFO-LAX|      38|
+-------+--------+

Num itin - one-way

In [104]:
count_num_records_to_cutoff(ow_out_fi_one_str_agg, cutoff, "itin")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
| market|num_itin|
+-------+--------+
|ATL-EWR|      19|
|EWR-CDG|      24|
|EWR-LHR|      11|
|JFK-LAX|      24|
|JFK-LHR|      15|
|LAX-EWR|      15|
|LAX-JFK|      32|
|LAX-SFO|      31|
|LGA-MIA|      11|
|LHR-EWR|      11|
|LHR-JFK|       4|
|LHR-LAX|      24|
|OAK-LAS|       6|
|SFO-LAX|      35|
+-------+--------+

Num itin - round-trip, out-bound

In [105]:
count_num_records_to_cutoff(rt_out_fi_per_leg_agg, cutoff, "itin")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
| market|num_itin|
+-------+--------+
|ATL-EWR|      40|
|EWR-CDG|      46|
|EWR-LHR|      75|
|JFK-LAX|      33|
|JFK-LHR|      69|
|LAX-EWR|      76|
|LAX-JFK|      36|
|LAX-SFO|      42|
|LGA-MIA|      20|
|LHR-EWR|      49|
|LHR-JFK|      54|
|LHR-LAX|     169|
|OAK-LAS|      30|
|SFO-LAX|      42|
+-------+--------+

Num itin - round-trip, out+in

In [106]:
count_num_records_to_cutoff(rt_fi_one_str_agg, cutoff, "itin")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
| market|num_itin|
+-------+--------+
|ATL-EWR|     481|
|EWR-CDG|     675|
|EWR-LHR|    2168|
|JFK-LAX|     407|
|JFK-LHR|    1059|
|LAX-EWR|    1181|
|LAX-JFK|     426|
|LAX-SFO|     487|
|LGA-MIA|     163|
|LHR-EWR|     653|
|LHR-JFK|     660|
|LHR-LAX|    3945|
|OAK-LAS|     189|
|SFO-LAX|     530|
+-------+--------+

## reverse: top X = Y %

In [45]:
def top_num_records_to_cutoff(df, top_n):
    w3 = Window.partitionBy("market").orderBy('cum_pct')
    window_df = df.withColumn("row_num", F.row_number().over(w3))
    
    filt_df = window_df.filter(F.col("row_num") <= top_n)
    # could add a collect_list agg here 
    final_result = filt_df.groupBy("market").agg(F.max("cum_pct").alias("pct_lowest_fares"))
    final_result.orderBy("market").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Num itin - one-way

In [46]:
num = 50

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
top_num_records_to_cutoff(ow_out_fi_one_str_agg, num)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
| market|  pct_lowest_fares|
+-------+------------------+
|ATL-EWR|0.9872548897531371|
|EWR-CDG|0.9714459694733725|
|EWR-LHR| 0.991232873065948|
|JFK-LAX|0.9807788155956191|
|JFK-LHR|0.9962724492941113|
|LAX-EWR|0.9732849236633097|
|LAX-JFK|0.9732303440851725|
|LAX-SFO|0.9665233613782483|
|LGA-MIA|0.9974719748789513|
|LHR-EWR|0.9999662973340323|
|LHR-JFK|0.9984920781654234|
|LHR-LAX|0.9584603871371288|
|OAK-LAS|0.9993746846469986|
|SFO-LAX|0.9521480233388164|
+-------+------------------+

Num legs - one-way

In [51]:
top_num_records_to_cutoff(ow_out_fi_per_leg_agg, num)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
| market|  pct_lowest_fares|
+-------+------------------+
|ATL-EWR|0.9803800196475481|
|EWR-CDG|0.9699616361756168|
|EWR-LHR|0.9962520570627592|
|JFK-LAX| 0.965064610426311|
|JFK-LHR|  0.99654331736802|
|LAX-EWR| 0.947163290885227|
|LAX-JFK|0.9533860238944117|
|LAX-SFO|0.9658425706989779|
|LGA-MIA|0.9949153966451393|
|LHR-EWR|0.9999863367508157|
|LHR-JFK| 0.999050456226621|
|LHR-LAX|0.9687831973805302|
|OAK-LAS|0.9975421362144964|
|SFO-LAX|0.9504294396601616|
+-------+------------------+

Carriers

In [52]:
num = 8

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
top_num_records_to_cutoff(ow_out_cxr_crude_agg, num)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
| market|  pct_lowest_fares|
+-------+------------------+
|ATL-EWR| 0.999999999999998|
|EWR-CDG|0.9847512151860822|
|EWR-LHR|0.9995539732840663|
|JFK-LAX| 1.000000000000002|
|JFK-LHR|0.9974988870926151|
|LAX-EWR|1.0000000000000029|
|LAX-JFK| 1.000000000000001|
|LAX-SFO|0.9999999999999994|
|LGA-MIA|1.0000000000000013|
|LHR-EWR|1.0000000000000013|
|LHR-JFK|0.9991790244635154|
|LHR-LAX|0.9843805842054117|
|OAK-LAS| 1.000000000000001|
|SFO-LAX|0.9999999999999981|
+-------+------------------+

In [54]:
top_num_records_to_cutoff(rt_out_cxr_crude, num)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
| market|    pct_lowest_fares|
+-------+--------------------+
|ATL-EWR|0.007121860338208427|
|EWR-CDG|0.004824391714101888|
|EWR-LHR| 0.00706630454013813|
|JFK-LAX|0.006077636122111586|
|JFK-LHR|0.004519228747661...|
|LAX-EWR|0.044480279618341285|
|LAX-JFK|0.009122535049097552|
|LAX-SFO|0.006372895370040743|
|LGA-MIA|0.012548261322763347|
|LHR-EWR|0.008032519048467517|
|LHR-JFK|0.003894893106871...|
|LHR-LAX|0.005810493021404...|
|OAK-LAS|0.014641903360541303|
|SFO-LAX|0.006205120934300121|
+-------+--------------------+

In [55]:
top_num_records_to_cutoff(rt_in_cxr_crude, num)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
| market|    pct_lowest_fares|
+-------+--------------------+
|ATL-EWR|0.007121860338208427|
|EWR-CDG| 0.00482597615525814|
|EWR-LHR|0.007468572609879005|
|JFK-LAX|0.006077749695275943|
|JFK-LHR|0.004520409020787932|
|LAX-EWR| 0.04447893463127946|
|LAX-JFK|0.009122208661439264|
|LAX-SFO|0.006372895370040743|
|LGA-MIA|0.012548261322763347|
|LHR-EWR|0.008283242014615066|
|LHR-JFK|0.003954950381828425|
|LHR-LAX|0.005845304618325546|
|OAK-LAS|0.014654595107017679|
|SFO-LAX|0.006205128719877742|
+-------+--------------------+